In [23]:
from PIL import Image
import os
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import shutil

def strip_iccp(png_path, save_path):
    with Image.open(png_path) as img:
        temp_path = save_path + ".tmp"
        img.save(temp_path, format="PNG", icc_profile=None)
    shutil.move(temp_path, save_path)
    print(temp_path, save_path)


In [24]:
def has_iccp_chunk(path):
    with open(path, "rb") as f:
        data = f.read()

    # PNG chunks start after 8-byte signature
    pos = 8
    while pos < len(data):
        length = int.from_bytes(data[pos:pos+4], 'big')
        chunk_type = data[pos+4:pos+8]
        if chunk_type == b"iCCP":
            return True
        pos += length + 12  # 4 (length) + 4 (chunk type) + data + 4 (CRC)
    return False

In [37]:
folder = "../datasets/anime/illustrateData_multiscale_sub"

for file in tqdm(os.listdir(folder)):
    if not file.endswith(".jpg"):
        continue
    path = os.path.join(folder, file)
    if has_iccp_chunk(path):
        print(f"⚠️ iCCP found in: {file} — stripping")
        strip_iccp(path, path)
        if not has_iccp_chunk(path):
            print(f"✅ Cleaned: {file}")
        else:
            print(f"❌ Still contains iCCP: {file}")


100%|██████████| 41178/41178 [00:00<00:00, 4838984.93it/s]
